In [0]:
import numpy as np
import pandas as pd

In [0]:
# HERE IS READING FROM READING FROM STORAGE ACCOUNT
storage_account_key = "wgbe0Fzs4W3dPNc35dp//uumz+SPDXVLLGu0mNaxTs2VLHCCPnD7u79PYt4mKeSFboqMRnZ+s+ez+ASty+k+sQ=="
storage_account_name = "factoredatathon2024"
container_name = "gold"

spark.conf.set(
    f"fs.azure.account.key.{storage_account_name}.blob.core.windows.net",
    f"{storage_account_key}"
)

file_path = f"wasbs://{container_name}@{storage_account_name}.blob.core.windows.net/gkg/themesSortedGold.csv"
df = spark.read.format("csv").option("header", "true").load(file_path)
df = df.dropna(subset=["THEMES_EXPLODED"])

In [0]:
### HERE YOU CAN READ FROM AZURE SQL
# Define the JDBC URL
jdbc_hostname = "factoredata2024.database.windows.net"
jdbc_port = 1433
jdbc_database = "dactoredata2024"
jdbc_url = f"jdbc:sqlserver://{jdbc_hostname}:{jdbc_port};database={jdbc_database}"

# Define the connection properties
connection_properties = {
    "user": "factoredata2024admin",
    "password": "mdjdmliipo3^%^$5mkkm63",
    "driver": "com.microsoft.sqlserver.jdbc.SQLServerDriver"
}

# Define your SQL query
#sql_query = "(SELECT THEMES_EXPLODED, count FROM [gkg].[THEMES] WHERE THEMES_EXPLODED != '' AND count > 100) AS tmp"
sql_query = "(SELECT THEMES_EXPLODED, count FROM [gkg].[THEMES] WHERE THEMES_EXPLODED != '') AS tmp"

# Load data from Azure SQL Database into a DataFrame
df = spark.read.jdbc(url=jdbc_url, table=sql_query, properties=connection_properties)

In [0]:
df.show()

In [0]:
themes = df.toPandas()


In [0]:
themes_list = list(themes['THEMES_EXPLODED'])
themes_list

In [0]:
themes.head()

## TfidfVectorizer + Kmeans Approach


In [0]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans

vectorizer = TfidfVectorizer(stop_words='english', max_features=5000)
X = vectorizer.fit_transform(themes_list)  # themes_list es la lista de temas

kmeans = KMeans(n_clusters=10, init='k-means++', max_iter=500, n_init=10, random_state=42)
kmeans.fit(X)

themes['Cluster'] = kmeans.labels_

In [0]:
themes['Cluster'].value_counts()

In [0]:
from sklearn.manifold import TSNE
import matplotlib.pyplot as plt

# Apply t-SNE to reduce dimensionality to 2D
tsne = TSNE(n_components=2, random_state=42)
X_tsne = tsne.fit_transform(X.toarray())

# Convert the t-SNE result into a DataFrame for easier manipulation
tsne_df = pd.DataFrame(X_tsne, columns=['Dim1', 'Dim2'])
tsne_df['Cluster'] = themes['Cluster']

# Visualize the clusters
plt.figure(figsize=(10, 8))
scatter = plt.scatter(tsne_df['Dim1'], tsne_df['Dim2'], c=tsne_df['Cluster'], cmap='viridis', s=10, alpha=0.6)

# Add legend with clusters
plt.legend(*scatter.legend_elements(), title="Clusters")
plt.title('Clusters Visualized with t-SNE')
plt.xlabel('Dim1')
plt.ylabel('Dim2')
plt.show()


In [0]:
themes.head()

In [0]:
themes['Cluster'].value_counts()


In [0]:
# plot clusters for all themes

# Apply t-SNE to reduce dimensionality to 2D
tsne = TSNE(n_components=2, random_state=42)
X_tsne = tsne.fit_transform(np.array(all_vectors))

# Convert the t-SNE result into a DataFrame for easier manipulation
tsne_df = pd.DataFrame(X_tsne, columns=['Dim1', 'Dim2'])
tsne_df['Cluster'] = labels

# Visualize the clusters
plt.figure(figsize=(10, 8))
scatter = plt.scatter(tsne_df['Dim1'], tsne_df['Dim2'], c=tsne_df['Cluster'], cmap='viridis', s=10, alpha=0.6)

# Add legend with clusters
plt.legend(*scatter.legend_elements(), title="Clusters")
plt.title('Clusters Visualized with t-SNE')
plt.xlabel('Dim1')
plt.ylabel('Dim2')
plt.show()


In [0]:
len(themes.query('Cluster == 0'))

In [0]:
# df.count()
# 54912 themes

## Word2Vec Approach


In [0]:
%pip install nltk

In [0]:
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer

In [0]:
nltk.download('punkt')
nltk.download('stopwords')

In [0]:
codes = themes_list.copy()

# Decomposition by the separator "_"
tokenized_codes = [code.split('_') for code in codes]

print(tokenized_codes)


In [0]:
from nltk.corpus import stopwords
# List of stopwords and words to remove

# Download NLTK stopwords if you don't have them already
nltk.download('stopwords')

# Get the list of English stopwords and add 'TAX'
stopwords_list = set(word.upper() for word in stopwords.words('english')) 
custom_stopwords = {'TAX', 'GENERAL', 'WB', 'EPU', 'USPEC', 'CRISISLEX'}

# Combine NLTK stopwords with custom stopwords
all_stopwords = stopwords_list.union(custom_stopwords)

# Filter lists by removing stopwords and the word 'TAX'
filtered_codes = [[word for word in code if word.upper() not in all_stopwords] for code in tokenized_codes]

print(filtered_codes)


In [0]:
%pip install gensim

In [0]:
from gensim.models import Word2Vec

# Training the Word2Vec model
model = Word2Vec(sentences=filtered_codes, vector_size=100, window=5, min_count=1, sg=0)


In [0]:
def get_vector_for_code(code, model):
    # Get word vectors for each word in the code
    vectors = [model.wv[word] for word in code if word in model.wv]
    if len(vectors) == 0:
        return np.zeros(model.vector_size)
    return np.mean(vectors, axis=0)

# Get vectors for all codes
vectors = np.array([get_vector_for_code(code, model) for code in filtered_codes])


In [0]:
# Number of clusters
n_clusters = 3  # Adjust this value as needed

# Apply K-Means
kmeans = KMeans(n_clusters=n_clusters, random_state=42)
clusters = kmeans.fit_predict(vectors)

# Add clusters to the codes
clustered_codes = list(zip(filtered_codes, clusters))


In [0]:
import seaborn as sns

# Dimensionality reduction to 2D with t-SNE
tsne = TSNE(n_components=2, random_state=42)
tsne_result = tsne.fit_transform(vectors)

df = pd.DataFrame(tsne_result, columns=['Dimension 1', 'Dimension 2'])
df['Cluster'] = clusters

# Visualization
plt.figure(figsize=(10, 8))
sns.scatterplot(x='Dimension 1', y='Dimension 2', hue='Cluster', palette='viridis', data=df, s=100, alpha=0.7)
plt.title('Cluster Visualization using t-SNE')
plt.legend(title='Cluster')
plt.show()



In [0]:
from sklearn.decomposition import PCA
from mpl_toolkits.mplot3d import Axes3D

# Apply PCA to reduce to 3D
pca = PCA(n_components=3)
reduced_vectors_3d = pca.fit_transform(vectors)

# Create the 3D plot
fig = plt.figure(figsize=(10, 7))
ax = fig.add_subplot(111, projection='3d')
scatter = ax.scatter(reduced_vectors_3d[:, 0], reduced_vectors_3d[:, 1], reduced_vectors_3d[:, 2], c=clusters, cmap='viridis', marker='o')

# Add a color bar to show cluster labels
plt.colorbar(scatter)

# Add labels and title
ax.set_xlabel('Component 1')
ax.set_ylabel('Component 2')
ax.set_zlabel('Component 3')
ax.set_title('3D Cluster Visualization')

# Show the plot
plt.show()


In [0]:
from collections import Counter
import pandas as pd

# Create a DataFrame with codes and their respective clusters
df = pd.DataFrame({'Code': filtered_codes, 'Cluster': clusters})

# Create a dictionary to count words in each cluster
cluster_word_counts = {}

for cluster_id in range(n_clusters):
    # Get all codes in the cluster
    cluster_codes = df[df['Cluster'] == cluster_id]['Code']
    
    # Flatten the list of lists of codes into a single list of words
    all_words = [word for code in cluster_codes for word in code]
    
    # Count the words
    word_counts = Counter(all_words)
    
    # Save the count in the dictionary
    cluster_word_counts[cluster_id] = word_counts

# Display the most common words in each cluster
for cluster_id, word_counts in cluster_word_counts.items():
    print(f"Cluster {cluster_id}:")
    for word, count in word_counts.most_common(10):  # Show the top 10 most common words
        print(f"  {word}: {count}")
    print()


In [0]:
import pandas as pd
from collections import Counter

# Create a DataFrame with codes and their respective clusters
df = pd.DataFrame({'Code': filtered_codes, 'Cluster': clusters})

# Create a dictionary mapping clusters to desired names
cluster_names = {
    0: 'ECONOMIC',   # Adjust cluster numbers as per your clustering results
    1: 'POLITICAL',
    2: 'SOCIAL'
}

# Replace cluster identifiers with names
df['Cluster_Name'] = df['Cluster'].map(cluster_names)

# Display the DataFrame with cluster names
print(df.head())

# Optional: Count words in each newly named cluster
cluster_word_counts = {}
for cluster_name in cluster_names.values():
    # Get all codes in the cluster
    cluster_codes = df[df['Cluster_Name'] == cluster_name]['Code']
    
    # Flatten the list of lists of codes into a single list of words
    all_words = [word for code in cluster_codes for word in code]
    
    # Count the words
    word_counts = Counter(all_words)
    
    # Save the count in the dictionary
    cluster_word_counts[cluster_name] = word_counts

# Display the most common words in each newly named cluster
for cluster_name, word_counts in cluster_word_counts.items():
    print(f"Cluster '{cluster_name}':")
    for word, count in word_counts.most_common(10):  # Show the top 10 most common words
        print(f"  {word}: {count}")
    print()


In [0]:
import pandas as pd

# Create a DataFrame with themes and their corresponding clusters
df_clusters = pd.DataFrame({'THEMES_EXPLODED': themes_list, 'Cluster': clusters})

# Create a dictionary to map cluster IDs to descriptive names
cluster_names = {
    0: 'ECONOMIC',   # Adjust cluster numbers according to your clustering results
    1: 'SOCIAL',
    2: 'POLITICAL'
}

# Map cluster IDs to names
df_clusters['Cluster_Name'] = df_clusters['Cluster'].map(cluster_names)

# Create a mapping of THEMES_EXPLODED to Cluster_Name
themes_mapping = df_clusters.set_index('THEMES_EXPLODED')['Cluster_Name'].to_dict()

# Add the Cluster_Name column to the original 'themes' DataFrame
themes['Cluster_Name'] = themes['THEMES_EXPLODED'].map(themes_mapping)

# Display the DataFrame with the new column
print(themes.head())



##Labeling improved


In [0]:
# Function to obtain the average vector for a code
def get_code_vector(code):
    parts = code.split('_')
    vectors = [model.wv[part] for part in parts if part in model.wv]
    if vectors:
        return np.mean(vectors, axis=0)
    else:
        return np.zeros(model.vector_size)  # Zero vector if no tokens are in the vocabulary

# Example usage
vector_soc_generalcrime = get_code_vector('SOC_GENERALCRIME')
vector_arrest = get_code_vector('ARREST')

In [0]:
df = pd.DataFrame(tsne_result, columns=['Dimension 1', 'Dimension 2'])
df['Cluster'] = clusters

# Visualization
plt.figure(figsize=(10, 8))
sns.scatterplot(x='Dimension 1', y='Dimension 2', hue='Cluster', palette='viridis', data=df, s=100, alpha=0.7)
plt.title('Visualización de Clusters usando t-SNE')
plt.legend(title='Cluster')
plt.show()

In [0]:
from sklearn.cluster import KMeans

# Suppose `all_vectors` is a list of all the vectors for your codes
all_vectors = [
    get_code_vector(code) for code in codes
]

# Number of clusters (adjust as needed)
num_clusters = 5

# Training the K-means model
kmeans = KMeans(n_clusters=num_clusters, random_state=0).fit(all_vectors)

# Assigning labels to each code
labels = kmeans.labels_

print(labels)


In [0]:
labels_df = pd.DataFrame(labels, columns=['Cluster'])

# Count the values in each cluster
label_counts = labels_df['Cluster'].value_counts()

# Display the results
label_counts


In [0]:
from sklearn.manifold import TSNE
import matplotlib.pyplot as plt

# Assume that `all_vectors` is a list of all vectors for your codes
all_vectors = [
    get_code_vector(code) for code in codes
]

# Apply t-SNE to reduce dimensionality
tsne = TSNE(n_components=2, random_state=0)
reduced_vectors = tsne.fit_transform(all_vectors)

In [0]:
# Assume `labels` is a list of cluster labels for each code
labels = kmeans.labels_

# Create the plot
plt.figure(figsize=(10, 7))
scatter = plt.scatter(reduced_vectors[:, 0], reduced_vectors[:, 1], c=labels, cmap='viridis', marker='o')

# Add a color bar to show the cluster labels
plt.colorbar(scatter)

# Add labels and title
plt.xlabel('Component 1')
plt.ylabel('Component 2')
plt.title('Cluster Visualization with t-SNE')

# Show the plot
plt.show()


In [0]:
from sklearn.decomposition import PCA
from mpl_toolkits.mplot3d import Axes3D

# Apply PCA to reduce to 3D
pca = PCA(n_components=3)
reduced_vectors_3d = pca.fit_transform(all_vectors)

# Create the 3D plot
fig = plt.figure(figsize=(10, 7))
ax = fig.add_subplot(111, projection='3d')
scatter = ax.scatter(reduced_vectors_3d[:, 0], reduced_vectors_3d[:, 1], reduced_vectors_3d[:, 2], c=labels, cmap='viridis', marker='o')

# Add a color bar to show the cluster labels
plt.colorbar(scatter)

# Add labels and title
ax.set_xlabel('Component 1')
ax.set_ylabel('Component 2')
ax.set_zlabel('Component 3')
ax.set_title('3D Cluster Visualization')

# Show the plot
plt.show()


In [0]:
from collections import Counter
import numpy as np

# Suppose `tokenized_codes` is the list of lists of tokens obtained earlier
# and `labels` is the list of cluster labels for each code

# Create a dictionary to store the words by cluster
cluster_words = {i: [] for i in set(labels)}

# Populate the dictionary with the words from each cluster
for code, label in zip(tokenized_codes, labels):
    cluster_words[label].extend(code)

# Count the frequency of each word in each cluster
cluster_word_counts = {cluster: Counter(words) for cluster, words in cluster_words.items()}

# Find the 5 most frequent words in each cluster
top_words = {cluster: counter.most_common(5) for cluster, counter in cluster_word_counts.items()}

# Print the results
for cluster, words in top_words.items():
    print(f'Cluster {cluster}:')
    for word, freq in words:
        print(f'  "{word}" with frequency {freq}')
    print()
